In [1]:
import pandas as pd
import pgeocode
from config import gkey
import requests
import json
import pymongo
import numpy as np

In [2]:
zips_df=pd.read_csv('sf_zips.csv')

In [3]:
zips_df["Latitude"] = ""
zips_df["Longitude"] = ""

In [4]:
tool = pgeocode.Nominatim('US')

In [5]:
tool_table=tool.query_postal_code(95822)

In [6]:
tool_table["latitude"]

38.5091

In [7]:
for index, row in zips_df.iterrows():
    zipcode = row['ZIPCode']
    tool_table=tool.query_postal_code(zipcode)
    lat=tool_table["latitude"]
    long=tool_table["longitude"]
    zips_df.loc[index, 'Latitude']=lat
    zips_df.loc[index, 'Longitude']=long
    

In [8]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'classDB' database in Mongo
db = client.locations_mdb

In [26]:
api_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in zips_df.iterrows():
    target_coordinates = f"{zips_df.loc[index, 'Latitude']},{zips_df.loc[index, 'Longitude']}" 
    target_radius = 50000
    target_type = "lodging"

# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey}

# run a request using our params dictionary
    response = requests.get(api_url, params=params).json()
    
    
    db.locations.insert_one(
        {
            'zip_code': row['ZIPCode'],
            'type': row['Type'],
            'county': row['County'],
            'population': row['Population'],
            'area_codes': row['Area Code(s)'],
            'latitude': row['Latitude'],
            'longitude': row['Longitude'],
            'hotels': response["results"]
        }
    )

In [18]:
#print(db.locations.find({"rating":3})
#print(db.locations.find_one())
for x in db.locations.find({"zip_code":94102}):
    print(1)

1


In [ ]:
#import json
#from pymongo import MongoClient
#collection_locations = db['locations']

#with open('currencies.json') as f:
#    file_data = json.load(f)

#locations.insert_one(file_data)

with open('data.txt', 'w') as outfile:
    json.dump(response["results"], outfile)

In [27]:
for location in db.locations.find({"hotels.rating":4.1}):
    for hotel in location["hotels"]:
        if hotel["rating"] == 4.1:
            print(json.dumps(hotel, sort_keys=True, indent=2))
            

{
  "business_status": "OPERATIONAL",
  "geometry": {
    "location": {
      "lat": 37.7867731,
      "lng": -122.4112871
    },
    "viewport": {
      "northeast": {
        "lat": 37.7882410802915,
        "lng": -122.4098094197085
      },
      "southwest": {
        "lat": 37.7855431197085,
        "lng": -122.4125073802915
      }
    }
  },
  "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
  "id": "3344890deedcb97b1c2d64814f92a02510ba39c8",
  "name": "The Clift Royal Sonesta Hotel",
  "opening_hours": {
    "open_now": true
  },
  "photos": [
    {
      "height": 785,
      "html_attributions": [
        "<a href=\"https://maps.google.com/maps/contrib/114937580614387417622\">The Clift Royal Sonesta Hotel</a>"
      ],
      "photo_reference": "CmRaAAAAnSMZSipY_0oTFIK99cIp9Ye39PWNp-oJCgi1xZfmrxz9WfkV635xuIhmtgR-01pEYOYDoLNa8jP9pwU3dBq0W1lP-rY-hSPe9X_iOjL09wQKN954lEdhOwNneX_ZsZo4EhDHSd-pEbkLdVAHIT8zx-JPGhRsmBdvcnzZ0J4QdnRMry8l6pMyJg",
      "width": 